In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv
/kaggle/input/new-york-city-taxi-fare-prediction/GCP-Coupons-Instructions.rtf
/kaggle/input/new-york-city-taxi-fare-prediction/train.csv
/kaggle/input/new-york-city-taxi-fare-prediction/test.csv


In [2]:
import pandas as pd

In [3]:
train_df = pd.read_csv('../input/new-york-city-taxi-fare-prediction/train.csv',nrows=1000000)
test_df = pd.read_csv('../input/new-york-city-taxi-fare-prediction/test.csv')
# train_df.head()

In [4]:
train_df.shape

(1000000, 8)

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   key                1000000 non-null  object 
 1   fare_amount        1000000 non-null  float64
 2   pickup_datetime    1000000 non-null  object 
 3   pickup_longitude   1000000 non-null  float64
 4   pickup_latitude    1000000 non-null  float64
 5   dropoff_longitude  999990 non-null   float64
 6   dropoff_latitude   999990 non-null   float64
 7   passenger_count    1000000 non-null  int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 61.0+ MB


In [6]:
train_df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,1000000.000000,1000000.000000,1000000.000000,999990.000000,999990.000000,1000000.000000
mean,11.348079,-72.526640,39.929008,-72.527860,39.919954,1.684924
std,9.822090,12.057937,7.626154,11.324494,8.201418,1.323911
min,-44.900000,-3377.680935,-3116.285383,-3383.296608,-3114.338567,0.000000
25%,6.000000,-73.992060,40.734965,-73.991385,40.734046,1.000000
50%,8.500000,-73.981792,40.752695,-73.980135,40.753166,1.000000
75%,12.500000,-73.967094,40.767154,-73.963654,40.768129,2.000000
max,500.000000,2522.271325,2621.628430,45.581619,1651.553433,208.000000


In [7]:
print(len(train_df))
train_df = train_df[train_df.fare_amount > 0]
print(len(train_df))
train_df = train_df[train_df.passenger_count > 0]
print(len(train_df))
train_df = train_df.iloc[np.where((train_df.pickup_latitude >= -90) & (train_df.pickup_latitude <= 90))]
print(len(train_df))
train_df = train_df.iloc[np.where((train_df.dropoff_latitude >= -90) & (train_df.dropoff_latitude <= 90))]
print(len(train_df))
train_df = train_df.iloc[np.where((train_df.pickup_longitude >= -180) & (train_df.pickup_longitude <= 180))]
print(len(train_df))
train_df = train_df.iloc[np.where((train_df.dropoff_longitude >= -180) & (train_df.dropoff_longitude <= 180))]
print(len(train_df))

1000000
999933
996368
996356
996348
996337
996328


In [8]:
train_df.isna().sum()

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [8]:
# train_df.dropoff_longitude.fillna(train_df.dropoff_longitude.mode()[0],inplace=True)
# train_df.dropoff_latitude.fillna(train_df.dropoff_longitude.mode()[0],inplace=True)

In [9]:
train_df = train_df.astype({'pickup_datetime':'datetime64'})
test_df = test_df.astype({'pickup_datetime':'datetime64'})
train_df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1


In [10]:
train_df['month'] = train_df.pickup_datetime.dt.month
train_df['year'] = train_df.pickup_datetime.dt.year
train_df['hour'] = train_df.pickup_datetime.dt.hour
train_df['day_of_week_name'] = train_df.pickup_datetime.dt.day_name()
train_df['day_of_week'] = train_df.pickup_datetime.dt.dayofweek

test_df['month'] = train_df.pickup_datetime.dt.month
test_df['year'] = train_df.pickup_datetime.dt.year
test_df['hour'] = train_df.pickup_datetime.dt.hour
test_df['day_of_week_name'] = train_df.pickup_datetime.dt.day_name()
test_df['day_of_week'] = train_df.pickup_datetime.dt.dayofweek

In [11]:
train_df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,month,year,hour,day_of_week_name,day_of_week
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,6,2009,17,Monday,0
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,1,2010,16,Tuesday,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,8,2011,0,Thursday,3
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,4,2012,4,Saturday,5
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,3,2010,7,Tuesday,1


In [44]:
import geopy.distance
from tqdm import tqdm
tqdm.pandas()

train_df['dist_travelled'] = train_df.progress_apply(lambda x: round(geopy.distance.geodesic((x['pickup_latitude'],x['pickup_longitude']),(x['dropoff_latitude'],x['dropoff_longitude'])).km,2),axis=1)
test_df['dist_travelled'] = test_df.progress_apply(lambda x: round(geopy.distance.geodesic((x['pickup_latitude'],x['pickup_longitude']),(x['dropoff_latitude'],x['dropoff_longitude'])).km,2),axis=1)
train_df.head()

100%|██████████| 9914/9914 [00:02<00:00, 3805.44it/s]


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,month,year,hour,day_of_week_name,day_of_week,dist_travelled
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,6,2009,17,Monday,0,1.03
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,1,2010,16,Tuesday,1,8.44
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,8,2011,0,Thursday,3,1.39
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,4,2012,4,Saturday,5,2.80
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,3,2010,7,Tuesday,1,2.00


In [12]:
from sklearn.preprocessing import OrdinalEncoder
le = OrdinalEncoder()
train_df['month'] = le.fit_transform(train_df[['month']])
test_df['month'] = le.fit_transform(test_df[['month']])

In [13]:
from sklearn.model_selection import train_test_split

X = train_df.drop(columns=['key','fare_amount','pickup_datetime','day_of_week_name','dist_travelled'])
y = train_df.fare_amount
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [14]:
X_train.columns

Index(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'passenger_count', 'month', 'year', 'hour',
       'day_of_week'],
      dtype='object')

In [15]:
from xgboost import XGBRegressor

xgb = XGBRegressor()
xgb.fit(X_train,y_train)
y_pred_train = xgb.predict(X_train)
y_pred_test = xgb.predict(X_test)

In [16]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_train, y_pred_train))
print(mean_squared_error(y_test, y_pred_test))

15.097731158642485
18.871061076792635


In [17]:
print(mean_squared_error(y_train, y_pred_train,squared=False))
print(mean_squared_error(y_test, y_pred_test,squared=False))

3.885579899917448
4.344083456471875


In [48]:
final_df = pd.DataFrame()
final_df['key'] = test_df.key
final_df['fare_amount'] = xgb.predict(test_df.drop(columns=['key','pickup_datetime','day_of_week_name']))

In [50]:
final_df.to_csv('submission.csv',index=False)

In [18]:
!pip install mlfoundry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 833.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 10.8 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.6/183.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.7/410.7 kB 19.2 MB/s eta 0:00:00
  Created wheel for databricks-cli: filename=databricks_cli-0.17.3-py3-none-any.whl size=139102

In [20]:
import pickle

pickle.dump(xgb, open('xgb.pkl', 'wb'))

In [22]:
import mlfoundry as mlf
mlf.login()

Please get your API key from https://app.truefoundry.com/settings


Paste your API key and hit enter: ····················································


Writing API key at /root/.mlfoundry/credentials.netrc


True

In [23]:
client = mlf.get_client()
run = client.create_run(project_name="taxi-fare-prediction", run_name="xgb-model")

MlflowException: API request to https://auth.production.truefoundry.com/api/v1/oauth/api-keys/token failed with exception HTTPSConnectionPool(host='auth.production.truefoundry.com', port=443): Max retries exceeded with url: /api/v1/oauth/api-keys/token (Caused by ResponseError('too many 500 error responses'))